# Анализ результатов A/B-тестирования интернет-магазина

- Автор: Юлия прохорова
- Дата: 29.05.2025

**Цель:**
- провести оценку результатов A/B-теста.

**Задачи:**
- загрузить данные теста;
- проверить корректность его проведения;
- проанализировать полученные результаты.

**ТЗ:**
- название теста: interface_eu_test;
- группы: А (контрольная), B (новый интерфейс);
- гипотеза: упрощение интерфейса приведёт к тому, что в течение семи дней после регистрации в системе конверсия зарегистрированных пользователей в покупателей увеличится как минимум на три процентных пункта.

## Описание данных


`participants` - таблица участников тестов, содержит колонки:
- `user_id` — идентификатор пользователя;
- `group` — группа пользователя;
- `ab_test` — название теста;
- `device` — устройство, с которого происходила регистрация.

`events` - таблица о событиях, содержит колонки:
- `user_id` — идентификатор пользователя;
- `event_dt` — дата и время события;
- `event_name` — тип события;
- `details` — дополнительные данные о событии.

Дополнительная информация по столбцу `details`:
- `registration` (регистрация) — стоимость привлечения клиента;
- `purchase` (покупка) — стоимость покупки.

Импортируем библиотеки:

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest
import numpy as np
import statsmodels.stats.power as smp
import statsmodels.stats.proportion as smprop

Изучим данные:

In [7]:
display(participants.head())
participants.info()

,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB


In [8]:
display(events.head())
events.info()

,user_id,event_dt,event_name,details
0,GLOBAL,2020-12-01 00:00:00,End of Black Friday Ads Campaign,ZONE_CODE15
1,CCBE9E7E99F94A08,2020-12-01 00:00:11,registration,0.0
2,GLOBAL,2020-12-01 00:00:25,product_page,NaN
3,CCBE9E7E99F94A08,2020-12-01 00:00:33,login,NaN
4,CCBE9E7E99F94A08,2020-12-01 00:00:52,product_page,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787286 entries, 0 to 787285
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     787286 non-null  object        
 1   event_dt    787286 non-null  datetime64[ns]
 2   event_name  787286 non-null  object        
 3   details     249022 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 24.0+ MB


**Выводы:**
- данные соответствуют описанию;
- пропуски в столбце `details`. Оставим без изменений.


## По таблице `ab_test_participants` оценим корректность проведения теста

Выделим пользователей, участвующих в тесте, и проверим:
   - соответствие требованиям технического задания;
   - равномерность распределения пользователей по группам теста;
   - отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

In [9]:
# Проверяем распределение пользователей по группам
grouped_users=participants.groupby(['ab_test', 'group']).agg({'user_id': 'nunique'}).reset_index()
grouped_users

,ab_test,group,user_id
0,interface_eu_test,A,5383
1,interface_eu_test,B,5467
2,recommender_system_test,A,2747
3,recommender_system_test,B,928


In [10]:
# Рассчитываем процентную разницу между группами A и B
value_A = grouped_users[grouped_users['group'] == 'A']['user_id'].values[0]
value_B = grouped_users[grouped_users['group'] == 'B']['user_id'].values[0]
percentage_diff = 100 * (value_A - value_B) / value_A
print(f"Процентная разница между группами A и B: {percentage_diff:.2f}%")

Процентная разница между группами A и B: -1.56%


Кроме исследуемого теста, был еще параллельный тест arecommender_system_test. Посмотрим, насколько пересеклись их аудитории:

In [11]:
#  Получаем списки user_id для каждого теста
interface_users = set(participants[participants['ab_test'] == 'interface_eu_test']['user_id'])
recommender_users = set(participants[participants['ab_test'] == 'recommender_system_test']['user_id'])

# Находим пересечение пользователей
common_users = interface_users & recommender_users

# Выводим результаты
print(f"Всего уникальных пользователей в interface_eu_test: {len(interface_users)}")
print(f"Количество пользователей, участвующих в обоих тестах: {len(common_users)}")
print(f"Процент пересечения относительно interface_eu_test: {len(common_users)/len(interface_users)*100:.2f}%")

Всего уникальных пользователей в interface_eu_test: 10850
Количество пользователей, участвующих в обоих тестах: 887
Процент пересечения относительно interface_eu_test: 8.18%


Удалим пользователей участвующих в 2х тестах.

In [12]:
participants = participants[~participants['user_id'].isin(common_users)]

**Выводы:**
- соответствует ТЗ;
- распределение пользователей по группам равномерное (разница несущественная - 1,56%);
- есть пересечений с конкурирующим тестом (8,18% пользователей, участвуют одновременно в двух тестах). Данные пользователи для дальнейших исследований были удалены.

Проанализируем данные о пользовательской активности по таблице `ab_test_events`:
- оставим события, связанные с участвующими в изучаемом тесте пользователями.

In [15]:
# Объединим таблицы
merged_df = pd.merge(
    left=participants,
    right=events,
    how='left',
    on='user_id'
)
merged_df.head()

,user_id,group,ab_test,device,event_dt,event_name,details
0,0002CE61FF2C4011,B,interface_eu_test,Mac,2020-12-07 04:37:31,registration,-2.38
1,0002CE61FF2C4011,B,interface_eu_test,Mac,2020-12-07 04:37:49,login,NaN
2,0002CE61FF2C4011,B,interface_eu_test,Mac,2020-12-07 04:37:57,login,NaN
3,0002CE61FF2C4011,B,interface_eu_test,Mac,2020-12-07 04:38:54,login,NaN
4,0002CE61FF2C4011,B,interface_eu_test,Mac,2020-12-08 22:15:35,login,NaN


In [16]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92758 entries, 0 to 92757
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     92758 non-null  object        
 1   group       92758 non-null  object        
 2   ab_test     92758 non-null  object        
 3   device      92758 non-null  object        
 4   event_dt    92758 non-null  datetime64[ns]
 5   event_name  92758 non-null  object        
 6   details     24628 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 5.7+ MB


Определим горизонт анализа: рассчитаем время (лайфтайм) совершения события пользователем после регистрации и оставим только те события, которые были выполнены в течение первых семи дней с момента регистрации.

In [17]:
#  Найдем время регистрации для каждого пользователя
registration_times = merged_df[merged_df['event_name'] == 'registration'].groupby('user_id')['event_dt'].min()

# Объединим с основным датафреймом, чтобы добавить время регистрации к каждому событию
filtered_df = merged_df.merge(registration_times.rename('registration_dt'), 
                           how='left', 
                           on='user_id')

# Рассчитаем лайфтайм (разница между временем события и регистрации в днях)
filtered_df['lifetime'] = filtered_df['event_dt'] - filtered_df['registration_dt']

# Оставим только события, которые произошли в первые 7 дней после регистрации
filtered_df = filtered_df[filtered_df['lifetime'] <= '7 days 00:00:00'].copy()
filtered_df.sort_values('lifetime', ascending=False)

,user_id,group,ab_test,device,event_dt,event_name,details,registration_dt,lifetime
64172,AFD2C642DE1D5D2A,A,recommender_system_test,iPhone,2020-12-23 14:31:23,product_page,NaN,2020-12-16 14:31:37,6 days 23:59:46
38263,693922FA03FC6666,B,recommender_system_test,PC,2020-12-22 18:21:25,purchase,4.890000000000001,2020-12-15 18:21:55,6 days 23:59:30
38262,693922FA03FC6666,B,recommender_system_test,PC,2020-12-22 18:20:15,purchase,99.99,2020-12-15 18:21:55,6 days 23:58:20
90344,F9236DA883A6285C,A,interface_eu_test,Android,2020-12-21 10:11:00,product_page,NaN,2020-12-14 10:12:50,6 days 23:58:10
24921,44692DD8619119D1,A,interface_eu_test,Mac,2020-12-22 20:45:42,product_cart,NaN,2020-12-15 20:50:55,6 days 23:54:47
...,...,...,...,...,...,...,...,...,...
30061,524879CA4380D898,A,interface_eu_test,iPhone,2020-12-10 02:55:00,registration,-3.8,2020-12-10 02:55:00,0 days 00:00:00
66432,B6C207AB67D61922,A,interface_eu_test,Android,2020-12-19 09:49:40,registration,0.0,2020-12-19 09:49:40,0 days 00:00:00
66422,B6C0B0221D98E4FA,B,interface_eu_test,Android,2020-12-08 22:40:06,registration,0.0,2020-12-08 22:40:06,0 days 00:00:00
66417,B6B88E3240A47646,A,recommender_system_test,Mac,2020-12-20 06:49:44,registration,0.0,2020-12-20 06:49:44,0 days 00:00:00


Оценим достаточность выборки для получения статистически значимых результатов A/B-теста. Параметры:
- базовый показатель конверсии — 30%,
- мощность теста — 80%,
- достоверность теста — 95%.

Для оценки достаточности выборки в A/B-тесте используем библиотеку statsmodels.

In [52]:
# Заданные параметры
baseline_conversion = 0.30  # базовая конверсия (30%)
effect_size = 0.05  # минимальный детектируемый эффект (5%)
alpha = 0.05  # уровень значимости (5%, соответствует 95% достоверности)
power = 0.80  # мощность теста (80%)

# Расчет MDE (Minimum Detectable Effect) в абсолютных значениях
mde_absolute = baseline_conversion * effect_size

# Расчет размера выборки для одной группы (формула для пропорций)
sample_size = smp.tt_ind_solve_power(
    effect_size=smprop.proportion_effectsize(baseline_conversion, baseline_conversion + mde_absolute),
    alpha=alpha,
    power=power,
    ratio=1.0,  # равные размеры групп
    alternative='two-sided'  # двусторонний тест
)

print(f"Размер выборки для каждой группы: {round(sample_size)}")

Размер выборки для каждой группы: 14856


Рассчитаем для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [53]:
# Фильтруем только события 'purchase'
purchase_events = filtered_df[filtered_df['event_name'] == 'purchase']

# Группируем по 'group' и считаем:
# 1) Уникальных пользователей, сделавших покупку
# 2) Общее количество уникальных пользователей в каждой группе (из исходного DataFrame)

# 1. Уникальные пользователи с покупкой по группам
purchase_users_per_group = purchase_events.groupby('group')['user_id'].nunique().reset_index()
purchase_users_per_group.columns = ['group', 'users_with_purchase']

# 2. Общее количество уникальных пользователей по группам (из всего filtered_df)
total_users_per_group = filtered_df.groupby('group')['user_id'].nunique().reset_index()
total_users_per_group.columns = ['group', 'total_users']

# Объединяем результаты
result = pd.merge(
    purchase_users_per_group,
    total_users_per_group,
    on='group'
)

result['conversion_rate'] = result['users_with_purchase'] / result['total_users']
print(result)

  group  users_with_purchase  total_users  conversion_rate
0     A                 1937         7034         0.275377
1     B                 1645         5717         0.287738


**Вывод:** 
- конверсия в тестовой группе B выше на ~4,5 процентных пункта  по сравнению с контрольной группой A, т.о.эффект от изменений есть, более 3 запланированых процентов. Определим является ли он статистически значимым.

## Проведем оценку результатов A/B-тестирования

Проверим изменение конверсии подходящим статистическим тестом, учитывая все этапы проверки гипотез.

Для проверки статистической значимости различий в конверсии между группами A и B используем Z-тест для пропорций (так как сравниваем доли успехов в двух группах). 

Сформулируем гипотизы:
- H0: статистически значимых различий нет;
- H1: статистически значимые разлия есть.

In [18]:
# Конверсия
conv_A = 0.287  
conv_B = 0.303  

# Размеры групп 
n_A = 7034
n_B = 5717

# Количество успехов (покупок)
success_A = int(conv_A * n_A)
success_B = int(conv_B * n_B)

# Z-тест для пропорций
z_score, p_value = proportions_ztest(
    count=[success_A, success_B],
    nobs=[n_A, n_B],
    alternative='two-sided'  
)

print(f"Z-статистика = {z_score:.3f}")
print(f"p-value = {p_value:.4f}")

# Интерпретация
if p_value < 0.05:
    print("Отвергаем H₀: различия в конверсии статистически значимы.")
else:
    print("Не отвергаем H₀: статистически значимых различий нет.")

Z-статистика = -1.980
p-value = 0.0477
Отвергаем H₀: различия в конверсии статистически значимы.


**Итоговые выводы:**
- распределение пользователей по группам равномерное;
- удалены пользователи, участвующие сразу в 2х тестах;
- конверсия в тестовой группе B выше на ~4,5%, более ожидаемых 3%, результат статистически значемый.

**Рекомендации:**
- Рекомендуем использовать новую версию сайта.